In [1]:
!wget http://images.cocodataset.org/zips/train2014.zip
!unzip train2014.zip -d /content/images && rm train2014.zip

Streaming output truncated to the last 5000 lines.
 extracting: /content/images/train2014/COCO_train2014_000000408557.jpg  
 extracting: /content/images/train2014/COCO_train2014_000000013714.jpg  
 extracting: /content/images/train2014/COCO_train2014_000000194043.jpg  
 extracting: /content/images/train2014/COCO_train2014_000000219859.jpg  
 extracting: /content/images/train2014/COCO_train2014_000000278135.jpg  
 extracting: /content/images/train2014/COCO_train2014_000000141015.jpg  
 extracting: /content/images/train2014/COCO_train2014_000000280923.jpg  
 extracting: /content/images/train2014/COCO_train2014_000000200024.jpg  
 extracting: /content/images/train2014/COCO_train2014_000000435713.jpg  
 extracting: /content/images/train2014/COCO_train2014_000000249993.jpg  
 extracting: /content/images/train2014/COCO_train2014_000000424160.jpg  
 extracting: /content/images/train2014/COCO_train2014_000000142761.jpg  
 extracting: /content/images/train2014/COCO_train2014_000000532668.jpg  


In [ ]:
import os
import tensorflow as tf
import numpy as np
from tqdm import tqdm

In [2]:
input_path = "/content/images/train2014"
output_path = "/content/VQA_preprocessed/ImagesFeatures"

In [7]:
def process_img_data(file_loc):
    raw_img_data = tf.io.read_file(file_loc)
    decoded_img = tf.image.decode_jpeg(raw_img_data, channels=3)

    # Resizing the image to 224x224 pixels
    resized_img = tf.image.resize(decoded_img, (224, 224))

    # Preprocessing the image using VGG19 preprocessing function
    preprocessed_img = tf.keras.applications.vgg19.preprocess_input(resized_img)

    return preprocessed_img, file_loc

In [8]:
def create_feature_file_path(image_path, is_train, input_path, output_path):
    full_image_path = os.path.join(input_path, image_path)
    filename = full_image_path.split('/')[-1].split('.')[0]
    feature_filename = filename + '.npy'

    if not os.path.exists(output_path):
        os.mkdir(output_path)

    subdir = 'train_data' if is_train else 'test_data'
    subdir_path = os.path.join(output_path, subdir)
    if not os.path.exists(subdir_path):
        os.mkdir(subdir_path)

    return os.path.join(subdir_path, feature_filename)

In [9]:
def generate_image_features(image_paths, is_train, feature_extractor_model, input_path, output_path):

    unique_paths = list(set(image_paths))
    print("Total images to process: ", len(unique_paths))

    image_dataset = tf.data.Dataset.from_tensor_slices(unique_paths)
    image_dataset = image_dataset.map(process_img_data, num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(64)

    for batch_images, batch_paths in tqdm(image_dataset):
        batch_features = feature_extractor_model(batch_images)
        batch_features = tf.reshape(batch_features, (batch_features.shape[0], -1, batch_features.shape[3]))

        # Save the features as NumPy files
        for features, img_path in zip(batch_features, batch_paths):
            path_str = img_path.numpy().decode("utf-8")
            feature_file_path = create_feature_file_path(path_str, is_train, input_path, output_path)
            np.save(feature_file_path, features.numpy())

In [10]:
def print_train_image_paths(train_data, input_path):
    print(f"Number of training images: {len(train_data)}")

    for train_item in train_data:
        relative_img_path = train_item[0]
        full_img_path = os.path.join(input_path, relative_img_path)
        print(full_img_path)
        break

443757
/content/images/train2014/COCO_train2014_000000458752.jpg


In [11]:
def load_train_data(data_path, input_path):
    train_data = np.load(data_path, allow_pickle=True).tolist()
    full_img_paths = []
    for item in train_data:
        relative_img_path = item[0]
        filename = relative_img_path.split('/')[-1]
        full_img_path = os.path.join(input_path, 'train2014', filename)
        full_img_paths.append(full_img_path)
    return full_img_paths

def create_feature_extractor_model():
    # Load the VGG19 model without the top classification layers
    vgg19_model = tf.keras.applications.VGG19(include_top=False, weights='imagenet')
    # Get the input and the output of the last layer
    input_layer = vgg19_model.input
    output_layer = vgg19_model.layers[-1].output
    # Create a new model with the input and the output of the last layer
    feature_extractor_model = tf.keras.Model(input_layer, output_layer)
    print("Image model ready")
    return feature_extractor_model

def process_train_data(data_path, input_path, output_path):
    train_img_paths = load_train_data(data_path, input_path)
    feature_extractor = create_feature_extractor_model()
    generate_image_features(train_img_paths, is_train=True, feature_extractor_model=feature_extractor, input_path=input_path, output_path=output_path)
    print("Train image features extraction done.")

data_path = '/content/drive/MyDrive/DL_Project/VQA_preprocessed/train.npy'
input_path = '/content/images'
output_path = '/content/ImagesFeatures'
process_train_data(data_path, input_path, output_path)

80134624/80134624 [==============================] - 0s 0us/step
image model ready
total image# to preprocess:  82783


100%|██████████| 1294/1294 [13:21<00:00,  1.61it/s]

train image done.


In [20]:
!zip -r /content/ImagesFeatures.zip /content/VQA_preprocessed/ImagesFeatures


Streaming output truncated to the last 5000 lines.
  adding: content/VQA_preprocessed/ImagesFeatures/train/COCO_train2014_000000180123.npy (deflated 78%)
  adding: content/VQA_preprocessed/ImagesFeatures/train/COCO_train2014_000000047975.npy (deflated 85%)
  adding: content/VQA_preprocessed/ImagesFeatures/train/COCO_train2014_000000141003.npy (deflated 92%)
  adding: content/VQA_preprocessed/ImagesFeatures/train/COCO_train2014_000000378066.npy (deflated 87%)
  adding: content/VQA_preprocessed/ImagesFeatures/train/COCO_train2014_000000147311.npy (deflated 81%)
  adding: content/VQA_preprocessed/ImagesFeatures/train/COCO_train2014_000000521657.npy (deflated 86%)
  adding: content/VQA_preprocessed/ImagesFeatures/train/COCO_train2014_000000536458.npy (deflated 83%)
  adding: content/VQA_preprocessed/ImagesFeatures/train/COCO_train2014_000000317495.npy (deflated 88%)
  adding: content/VQA_preprocessed/ImagesFeatures/train/COCO_train2014_000000543797.npy (deflated 83%)
  adding: content/VQA_

In [21]:
from google.colab import drive
drive.mount('/content/drive')
!cp /content/ImagesFeatures.zip /content/drive/MyDrive


Mounted at /content/drive
